In [1]:
import random
# project_id = str(uuid.uuid4())
run_id = random.randint(100000, 999999)
print(run_id)

927947


In [3]:
a = "fd0cf79c-8118-43a7-8f0b-059e8f78227a".upper().replace('-','_')

In [4]:
model_name = "FD0CF79C_8118_43A7_8F0B_059E8F78227A_exp_001_run_0001"

'FD0CF79C_8118_43A7_8F0B_059E8F78227A'

In [2]:
! pip freeze | grep fosforml

fosforml==1.0.6


In [3]:
_exp_data = {
    "algorithm_category": "Snowparkml",
    "algorithm_type": "Classification",
    "algorithms": [
      {
        "algorithm_name": "snowflake.ml.modeling.xgboost.XGBClassifier"
      }
    ],
    "dataset_name": "EMPLOYEE",
    "experiment_id": "600b9f90-f431-4590-83c7-f38fe138dedc",
    "experiment_name": "New_Experiment_05",
    "monitor_run_id": "000000",
    "project_id": "68bce134-6d7a-4e24-9591-86266438acf9",
    "stored_procedure": "run_experiment",
    "target_column": "LEAVEORNOT"
  }


In [34]:
_exp_data = '''{
            "monitor_run_id": "16e3f179-bb70-4632-9a39-9a6fac4a5581",
            "project_id": "test12345",
            "experiment_id" : "exp6789", 
            "experiment_name":"My_run_listing_exp", 
            "stored_procedure":"run_experiment",
            "algorithm_category":"Snowparkml",
            "algorithm_type":"Classification",
            "dataset_name":"EMPLOYEE_10L",
            "target_column":"LEAVEORNOT",
            "algorithms":[{"algorithm_name": "snowflake.ml.modeling.tree.DecisionTreeClassifier"}]
        }'''

In [ ]:
# payload-1 (classification Airline Delay dataset)
exp_data = '''{{
"source":"EXP", 
"project_id":"0e0fb803-22db-4d88-9f2f-f6f75b6abcf0", 
"id":"7bbb5061-54d4-4862-8d47-7fbee388a4d1", 
"run_id":"{0}", 
"exp_name": "Final_recipe", 
"algorithm_type":"classification", 
"algo_details": {{"snowflake.ml.modeling.xgboost.XGBClassifier": null}}, 
"dataset": "AIRLINE_DEP_DELAY_10K", 
"target_column": "DEP_DEL15"}}'''.format(run_id)

In [3]:
# payload-2 (Regression Alcohol dataset)
exp_data = '''{{
"source":"EXP", 
"project_id":"0e0fb803-22db-4d88-9f2f-f6f75b6abcf0", 
"id":"7bbb5061-54d4-4862-8d47-7fbee388a4d1", 
"run_id":"{0}", 
"exp_name": "Final_recipe", 
"algorithm_type":"regression", 
"algo_details": {{"snowflake.ml.modeling.ensemble.RandomForestRegressor": null}}, 
"dataset": "ALCOHOL_DATA_10L", 
"target_column": "QUALITY"}}'''.format(run_id)

In [ ]:
# payload-3 (classification Employee dataset)
exp_data = '''{{
"source":"EXP", 
"project_id":"0e0fb803-22db-4d88-9f2f-f6f75b6abcf0", 
"id":"7bbb5061-54d4-4862-8d47-7fbee388a4d1", 
"run_id":"{0}", 
"exp_name": "Final_recipe", 
"algorithm_type":"classification", 
"algo_details": {{"snowflake.ml.modeling.linear_model.LogisticRegression": null}}, 
"dataset": "EMPLOYEE_10L", 
"target_column": "LEAVEORNOT"}}'''.format(run_id)

In [ ]:
# payload-4 (Regression Diamonds dataset)
exp_data = '''{{
"source":"EXP", 
"project_id":"0e0fb803-22db-4d88-9f2f-f6f75b6abcf0", 
"id":"7bbb5061-54d4-4862-8d47-7fbee388a4d1", 
"run_id":"{0}", 
"exp_name": "Final_recipe", 
"algorithm_type":"regression", 
"algo_details": {{"snowflake.ml.modeling.xgboost.XGBRegressor": null}}, 
"dataset": "DIAMONDS", 
"target_column": "PRICE"}}'''.format(run_id)

In [4]:
_exp_data

{'algorithm_category': 'Snowparkml',
 'algorithm_type': 'Classification',
 'algorithms': [{'algorithm_name': 'snowflake.ml.modeling.xgboost.XGBClassifier'}],
 'dataset_name': 'EMPLOYEE',
 'experiment_id': '600b9f90-f431-4590-83c7-f38fe138dedc',
 'experiment_name': 'New_Experiment_05',
 'monitor_run_id': '000000',
 'project_id': '68bce134-6d7a-4e24-9591-86266438acf9',
 'stored_procedure': 'run_experiment',
 'target_column': 'LEAVEORNOT'}

In [5]:
from fosforio.manager import get_conn_details_from_ds_name
from snowflake.snowpark.session import Session
import json

Connection manager service url initialised to http://fdc-project-manager:80/project-manager
If you need to update its value then update the variable CONNECTION_MANAGER_BASE_URL in os env.


In [6]:
def create_stage(session, stage_name="insight_exp"):
    try:
        session.sql(f"create or replace stage {stage_name}").collect()
        return f"@{stage_name}"
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex

        
def get_session(dataset, project_id):
    """
    Method creates snowflake session object.
    :return:
    """
    try:
        conn = get_conn_details_from_ds_name(dataset, project_id)
        region=conn["params"]["READER"]["region"] if conn["params"]["READER"]["cloudPlatform"] is None \
                    else conn["params"]["READER"]["region"]+"."+conn["params"]["READER"]["cloudPlatform"]
        account = conn['params']['READER']['accountId'] if region is None \
                    else conn['params']['READER']['accountId']+"."+region
        CONNECTION_PARAMETERS = {
            "account": account,
            "user":conn['params']['READER']['user'],
            "password": conn['params']['READER']['password'],
            "role": conn['params']['READER']['role'],
            "database": conn['params']['READER']['database'],
            "warehouse": conn['params']['READER']['wareHouse'],
            "schema": conn['params']['READER']['schema']
        }
        return Session.builder.configs(CONNECTION_PARAMETERS).create()
    except Exception as ex:
        print("Error while creating snowflake session", ex)
        raise ex

In [7]:
def run_experiment(session: Session, exp_data: dict) -> list:
    # Imports
    from snowflake.ml.modeling.pipeline import Pipeline
    from snowflake.ml.modeling.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
    from snowflake.ml.modeling.metrics import r2_score, accuracy_score, precision_score, roc_auc_score, \
        f1_score, recall_score, log_loss, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
    from snowflake.snowpark.functions import col, is_null, regexp_replace, when, lit
    from snowflake.snowpark.types import StringType
    from snowflake.snowpark.exceptions import SnowparkSQLException
    import importlib, sys, json, os, logging
    from snowflake.ml.registry.registry import Registry
    from fosforml import register_model 


    # variable for holding logs
    logs = []
    
    # function for accumulating logs
    def log_message(level: str, message: str):
        logs.append(f"{level}: {message}")
    
    
    # Functions used in stored proc
    def apply_data_cleansing(df):
        """
        Method handles null values in snowpark dataframe.
        :param:
        df: input dataframe
        :returns:
        df_cleaned: dataframe after null handling
        """
        # fillna - Unknown and 0
        schema_fields = df.schema.fields
        fill_values = {field.name: "UNKNOWN" if isinstance(field.datatype, StringType) else 0 for field in schema_fields}
        df_cleaned = df.fillna(fill_values)
        return df_cleaned


    def get_feature_columns(df):
        """
        Identifies the numerical and categorical features in dataset.
        Identifies features for label encoding and one hot encoding
        :param:
        df: input dataframe
        :returns:
        categorical_features: list of non-numerical feature columns
        numerical_features: list of numerical feature columns
        le_column_features: list of feature label encoder columns
        oh_column_features: list of feature one hot encoder columns
        """
        schema_fields = df.schema.fields
        features = df.columns
        features.remove(exp_details.get("target_column"))
        df_schema = session.sql(f"DESCRIBE TABLE {exp_details.get('dataset_name')}").collect()
        categorical_types = ['VARCHAR','CHAR','STRING','TEXT','BOOL']
        categorical_features = []  
        for row in df_schema:
            for typ in categorical_types:
                if typ in row['type']:
                    categorical_features.append(row['name'])
                    break
        numerical_features = list(set(features) - set(categorical_features))
        print(f"numerical_features:  {numerical_features}")
        print(f"categorical_features: {categorical_features}")
        log_message("INFO",f"numerical_features:  {numerical_features}")
        log_message("INFO",f"categorical_features: {categorical_features}")
        
        
        #identify columns for labelencoding and onehotencoding   
        le_column_features = categorical_features
        oh_column_features = []
        if len(categorical_features) >= 1:
            log_message("INFO",f"{categorical_features} columns are non numeric in feature dataset, encoding required.")
            for column in categorical_features:
                if df.select(df[column]).distinct().count() < 10:
                    oh_column_features.append(column)
            print(f"Columns identified to be encoded with label encoder: {le_column_features}")
            print
            log_message("INFO",f"Columns identified to be encoded with label encoder: {le_column_features}")
            log_message("INFO",f"Columns identified to be encoded with one hot encoder: {oh_column_features}")
        return categorical_features, numerical_features, le_column_features, oh_column_features


    def create_and_run_preprocessing(df, categorical_features, numerical_features, le_column_features, oh_column_features):
        """
        Based on different features column input creates preprocessing steps and run it on input dataframe
        :param:
        df: input dataframe
        categorical_features: list of non-numerical feature columns
        numerical_features: list of numerical feature columns
        le_column_features: list of feature label encoder columns
        oh_column_features: list of feature one hot encoder columns
        :returns:
        df_train: preprocessed train dataset
        df_test: preprocessed test dataset
        """
        #pipeline steps 
        log_message("INFO","Setting up preprocessing pipeline based on dataset")
        categorical_pp = {f'le_{column}':LabelEncoder(input_cols=column, output_cols=column) for column in le_column_features}
        if len(oh_column_features)>0:
            categorical_pp['oh_enc'] = OneHotEncoder(input_cols=oh_column_features, output_cols=oh_column_features, handle_unknown='ignore')
        numerical_pp = {
            'scaler': MinMaxScaler(input_cols=numerical_features, output_cols=numerical_features)
        }
        steps = [(key, categorical_pp[key]) for key in categorical_pp if categorical_pp[key]!=[]] + \
        [(key, numerical_pp[key]) for key in numerical_pp if numerical_features!=[]]
        log_message("INFO",f"Selected preprocesing steps: \n{steps}")   
            
        # Run preprocessing pipeline steps
        log_message("INFO","Running data preprocessing pipeline") 
        df = Pipeline(steps=steps).fit(df).transform(df)
        log_message("INFO",f"Transformed dataset: \n {df.show()}")
        df_train, df_test = df.random_split(weights=[0.8, 0.2], seed=0)
        for col_name in df_train.schema.names:
            new_col = col_name.replace('.','_')
            df_train = df_train.withColumnRenamed(col_name, new_col)
        for col_name in df_test.schema.names:
            new_col = col_name.replace('.','_')
            df_test = df_test.withColumnRenamed(col_name, new_col)
        return df_train, df_test


    def run_estimator(df_train, df_test, input_cols):
        """
        trains on df_train and creates model object for given algorithm/estimator.
        runs prediction function of model object on test dataset
        :param:
        df_train: input training dataframe
        df_test: input test dataframe
        input_cols: list of input feature names
        :returns:
        model: trained model object
        df_pred: output predict dataframe
        """
        for algorithm in exp_details.get("algorithms"):
            algorithm = algorithm["algorithm_name"].rsplit('.', 1)
            module = importlib.import_module(algorithm[0])
            log_message("INFO",f"----Running Algorithm {algorithm[1]}----")
            attr = getattr(module, algorithm[1])
            pipe = Pipeline(steps=[("algorithm", attr(input_cols=input_cols
                                                  , label_cols=[exp_details.get("target_column")]
                                                  , output_cols=['PREDICTIONS']))]
                   )
    
            # Fit the pipeline
            log_message("INFO",f"Running model pipeline {algorithm[1]}")
            model = pipe.fit(df_train)
     
            # Test the model
            log_message("INFO","Running prediction on model with test dataset")
            df_pred = model.predict(df_test)
            return model, df_pred, algorithm[1]

    
    def try_or(fn):
        try:
            out = fn()
            return out
        except:
            return None
    
        
    def eval_metrics(df_pred):
        log_message("INFO","Generating Metrices")
        if exp_details.get("algorithm_type") == 'Classification':
            return {
            'accuracy_score': try_or(lambda: accuracy_score(df=df_pred, y_true_col_names=exp_details.get("target_column"), y_pred_col_names='PREDICTIONS')),
            'f1_score': try_or(lambda: f1_score(df=df_pred, y_true_col_names=exp_details.get("target_column"), y_pred_col_names='PREDICTIONS')),
            'recall_score': try_or(lambda: recall_score(df=df_pred, y_true_col_names=exp_details.get("target_column"), y_pred_col_names='PREDICTIONS')),
            'precision_score': try_or(lambda: precision_score(df=df_pred, y_true_col_names=exp_details.get("target_column"), y_pred_col_names='PREDICTIONS')),
            'roc_auc': try_or(lambda: roc_auc_score(df=df_pred, y_true_col_names=exp_details.get("target_column"), y_score_col_names='PREDICTIONS')),
            'log_loss': try_or(lambda: log_loss(df=df_pred, y_true_col_names=exp_details.get("target_column"), y_pred_col_names='PREDICTIONS'))
            }
        else:
            return {
            'mean_absolute_percentage_error': try_or(lambda: mean_absolute_percentage_error(df=df_pred, y_true_col_names=exp_details.get("target_column"), y_pred_col_names='PREDICTIONS')),
            'r2_score': try_or(lambda: r2_score(df=df_pred, y_true_col_name=exp_details.get("target_column"), y_pred_col_name='PREDICTIONS')),
            'mean_absolute_error': try_or(lambda: mean_absolute_error(df=df_pred, y_true_col_names=exp_details.get("target_column"), y_pred_col_names='PREDICTIONS')),
            'mean_squared_error': try_or(lambda: mean_squared_error(df=df_pred, y_true_col_names=exp_details.get("target_column"), y_pred_col_names='PREDICTIONS'))
            }


#     def register_model(model, metrics_info, estimator):
#         log_message("INFO","Started: Registering model on snowflake")
#         reg = Registry(session=session)
        
#         clean = lambda x : x.replace("-","_")
#         project_id = clean(exp_details.get("project_id"))
#         exp_name = clean(exp_details.get("experiment_name"))
#         run_id = clean(exp_details.get("monitor_run_id"))
#         model_name = "Experiment_"+project_id+"_FDC_"+exp_name
#         mv = reg.log_model(model=model,
#                            model_name=model_name,
#                            comment=exp_details.get("description"),
#                            version_name='V_'+exp_details.get("monitor_run_id"),
#                            python_version="3.9.19",
#                            conda_dependencies=["xgboost","scikit-learn"],
#                            metrics={"model_metrics": metrics_info,
#                                     "dataset_details": {
#                                         "dataset_name": exp_details.get("dataset_name"),
#                                         "target_column": exp_details.get("target_column"),
#                                     },
#                                     "hyper_parameters": {'criterion': 'entropy', 'max_depth': None, 'max_features': 8, 'min_samples_leaf': 7},
#                                      "algorithm_type": exp_details.get("algorithm_type"),
#                                      "algorithm": estimator,                                   
#                                      "experiment_name": exp_details.get("experiment_name"),
#                                      "source": "Experiment",
#                                      "status": "Trained",
#                                      "run_id": exp_details.get("monitor_run_id"),
#                                      "experiment_id": exp_details.get("experiment_id")})
#         log_message("INFO","Registeration of model completed!!!")
#         return model_name
            
            
    def create_tags(session, exp_details):
        for key in exp_details.keys():
            tag = session.sql(f"CREATE TAG IF NOT EXISTS {key}")
            tag.show()
        if exp_details.get('algorithm_type')=='Classification':
            metric_names=["accuracy_score","precision_score","recall_score","f1_score","roc_auc","log_loss"]
        else:
            metric_names=["r2_score","mean_absolute_error","mean_squared_error","mean_absolute_percentage_error"]
        for name in metric_names:
            tag = session.sql(f"CREATE TAG IF NOT EXISTS {name}")
            tag.show()

        
    def set_tags(session, m_name, exp_details, metric_info):
        for key, value in exp_details.items():
            value = str(value).replace("'","\"")
            tag = session.sql(f"ALTER MODEL IF EXISTS {m_name} SET TAG {key}='{value}'")
            tag.show()
        for key, value in metric_info.items():
            value = str(value).replace("'","\"")
            tag = session.sql(f"ALTER MODEL IF EXISTS {m_name} SET TAG {key}='{value}'")
            tag.show()

    try:
        # loading experiment details
        exp_details=exp_data

        # creating user tags if not exist
        create_tags(session, exp_details)

        # Reading dataset
        log_message("INFO","Reading dataset features")
        data = session.table(exp_details.get("dataset_name"))

        # fillna
        data = apply_data_cleansing(data)

        # Identify feature columns
        categorical_features, numerical_features, le_column_features, oh_column_features = get_feature_columns(data)

        # Based on feature, do preprocessing
        data_train, data_test = create_and_run_preprocessing(data, categorical_features, numerical_features, le_column_features, oh_column_features)
        
        # Save Test and Train Dataset
        data_train.write.mode("overwrite").save_as_table("EMPLOYEE_TRAIN_CLEANED")
        data_train.write.mode("overwrite").save_as_table("EMPLOYEE_TEST_CLEANED")
        
        # Run model training and prediction
        input_cols = data_train.columns
        input_cols.remove(exp_details.get("target_column"))
        model, data_pred, estimator = run_estimator(data_train, data_test, input_cols)

        # Evaluate model metrices
#         metrics_info = eval_metrics(data_pred)
#         log_message("INFO", f"{metrics_info}")

        # Register model on snowflake registry
        model_name = register_model(
            model_obj=model,
            session=session,
            name="HR_CHURN_01",
            snowflake_df=data_pred,
            dataset_name=exp_details.get("dataset_name"),
            dataset_source="SnowflakeDataset",
            description="This is a test model",
            flavour="snowflake",
            model_type="classification",
            conda_dependencies=["xgboost","scikit-learn"]
        )
        print(reg_res)
#         model_name = register_model(model, metrics_info, estimator)
        log_message("INFO", f"{model_name}")

        # Set relevant tags to model object
#         set_tags(session, model_name, exp_details, metrics_info)

        return [{"Execution Logs:": "\n".join(logs),
                 "experiment_name":exp_details.get("experiment_name", "sample_experiment"),
                 "version":"V1",
                 "matrices":{"model_metrics": metrics_info, "project_id": exp_details.get("project_id"), "source": "EXP"},
                 "algorithm_type":exp_details.get("algorithm_type"),
                 "algorithms": exp_details.get("algorithms"),
                 "dataset_name": exp_details.get("dataset_name"),
                 "target_column": exp_details.get("target_column"),
                 "run_status": "SUCCESS",
                 "registry_model_name":"Experiment_"+exp_details.get("project_id")+"_FDC_"+exp_details.get("experiment_name")}]
    except Exception as ex:
        log_message("ERROR", f"{repr(ex)}")
        raise ex("Execution Logs: "+"\n".join(logs))

In [12]:
%%time
# Initilization
# logging.basicConfig(stream=sys.stdout, level=logging.INFO)
print("Creating Snowflake Session object...")
# exp_details=json.loads(_exp_data)
session = get_session(_exp_data.get("dataset_name"),"fd0cf79c-8118-43a7-8f0b-059e8f78227a")
stage = create_stage(session)
print("Session has been created !")

print("Creating stored procedure...")
session.custom_package_usage_config['enabled'] = True
session.custom_package_usage_config['force_push'] = True
# session.custom_package_usage_config["cache_path"] = stage
session.sproc.register(func=run_experiment,
                       name="run_experiment",
                       packages=["snowflake-snowpark-python",
                                 "fosforml",
                                 "snowflake-ml-python",
                                 "scikit-learn==1.3.2"
                                 ],
                       is_permanent=True,
                       stage_location=stage,
                       replace=True)
print("Stored procedure has been created successfully!")

# tagging session
print("Setting tag to session object: tag= ", "110003")
session.query_tag="110003"

print("Executing Procedure")
# procedure_response = session.call("run_experiment", _exp_data)
procedure_response = run_experiment(session, _exp_data)
print("Stored Procedure Executed Successfully !")
print(procedure_response)

#Log in mlflow
print("Logging in mlflow completed !")

Creating Snowflake Session object...
Session has been created !
Creating stored procedure...


The following packages are not available in Snowflake: ['scikit-learn==1.3.2', 'fosforml'].
Unable to load environments from remote path @insight_exp, creating a fresh environment instead.
ERROR: snowflake-connector-python 3.11.0 has requirement urllib3<2.0.0,>=1.21.1; python_version < "3.10", but you'll have urllib3 2.2.1 which is incompatible.
ERROR: mosaic-utils 1.0.2 has requirement scikit-learn==1.2.1; python_version >= "3.8", but you'll have scikit-learn 1.3.2 which is incompatible.
ERROR: fosforio 1.0.2 has requirement pandas==2.0.0, but you'll have pandas 2.2.2 which is incompatible.
ERROR: botocore 1.34.131 has requirement urllib3<1.27,>=1.25.4; python_version < "3.10", but you'll have urllib3 2.2.1 which is incompatible.
ERROR: jupyterlab 3.2.4 has requirement jupyter-server~=1.4, but you'll have jupyter-server 2.0.0a1 which is incompatible.
ERROR: jupyterlab-server 2.25.3 has requirement jsonschema>=4.18.0, but you'll have jsonschema 3.2.0 which is incompatible.
You should c

SnowparkSQLException: (1304): 01b59e07-0000-73b0-0000-576d075a55e2: 391546 (XX000): SQL compilation error: Cannot create a Python function with the specified packages. Please check your packages specification and try again. 'One or more package conflicts were detected.'. Hint: These packages are available in other python runtimes:["scikit-learn"->[3.8, 3.10, 3.11], "wheel==0.43.0"->[3.8, 3.10, 3.11], "numpy==1.26.4"->[3.10, 3.11], "frozenlist"->[3.8, 3.10, 3.11], "packaging==23.2"->[3.8, 3.10, 3.11], "catboost"->[3.8, 3.10, 3.11], "scipy==1.13.1"->[3.10, 3.11], "pandas==2.2.2"->[3.10, 3.11], "pytimeparse==1.1.8"->[3.8, 3.10, 3.11], "snowflake-ml-python==1.5.0"->[3.8, 3.10, 3.11], "pyjwt==2.8.0"->[3.8, 3.10, 3.11], "pyarrow"->[3.8, 3.10, 3.11], "jmespath==1.0.1"->[3.8, 3.10, 3.11], "snowflake-snowpark-python"->[3.8, 3.10, 3.11], "joblib==1.4.2"->[3.8, 3.10, 3.11], "absl-py==1.4.0"->[3.8, 3.10, 3.11], "asn1crypto==1.5.1"->[3.8, 3.10, 3.11], "urllib3==2.2.1"->[3.8, 3.10, 3.11], "wrapt"->[

Python 3.9.18
